In [6]:
import gurobipy as gp

In [7]:
source, content = gp.multidict({"A":3,"B":1,"C":2})
output, max_content, demand = gp.multidict({"X":[2.5,100],"Y":[1.5,200]})
pool = ["P"]
connections = gp.tuplelist([("A","P"),("B","P"),("P","X"),("P","Y"),("C","X"),("C","Y")])
flowCost = 1.5

In [8]:
model = gp.Model("pool")
model.params.nonConvex = 2
flow = {}
concentration = {}
for c in connections:
    flow[c] = model.addVar(name = "[%s->%s]" % (c[0],c[1]))
for p in pool:
    concentration[p] = model.addVar(name = "Concentration[%s]" % p)
    left = []
    right = []
    for c in connections:
        if p in c and c.index(p) == 1:
            left.append(c)
        elif p in c and c.index(p) == 0:
            right.append(c)
    model.addConstr(sum(flow[i] for i in left) == sum(flow[i] for i in right),name = "flowBalance")
    model.addConstr(sum(flow[i] * content[i[0]] for i in left) == concentration[p] * (sum(flow[i] for i in left)))
    model.addConstr(sum(flow[i] * content[i[0]] for i in left) == concentration[p] * (sum(flow[i] for i in right)))
for o in output:
    concentration[o] = model.addVar(name = "Concentration[%s]" % o)
    receive = []
    flowAmount = 0
    contentAmount = 0
    for c in connections:
        if o in c and c.index(o) == 1:
            receive.append(c)
    model.addConstr(sum(flow[i] for i in receive) >= demand[o],name = o + "capacity")
    for r in receive:
        if r[0] in source:
            contentAmount += content[r[0]]*flow[r]
            flowAmount += flow[r]
        elif r[0] in pool:
            contentAmount += concentration[r[0]]*flow[r]
            flowAmount += flow[r]
    model.addConstr(contentAmount == concentration[o]*flowAmount)
    model.addConstr(concentration[o] <= max_content[o])
model.setObjective(sum(flow[i]*flowCost for i in flow),gp.GRB.MINIMIZE)
model.optimize()

Set parameter NonConvex to value 2
Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 5 rows, 9 columns and 10 nonzeros
Model fingerprint: 0xa21de2ac
Model has 4 quadratic constraints
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  QMatrix range    [1e+00, 1e+00]
  QLMatrix range   [1e+00, 3e+00]
  Objective range  [2e+00, 2e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+00, 2e+02]
Presolve removed 2 rows and 0 columns

Continuous model is non-convex -- solving as a MIP

Presolve removed 2 rows and 0 columns
Presolve time: 0.00s
Presolved: 39 rows, 18 columns, 80 nonzeros
Presolved model has 8 bilinear constraint(s)
Variable types: 18 continuous, 0 integer (0 binary)

Root relaxation: objective 5.250000e+02, 13 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf

In [9]:
print("price = ",model.ObjVal)
for i in flow:
    if flow[i].X > 1e-6:
        print(flow[i].varName,flow[i].X)   

price =  600.0000000082532
[B->P] 100.00000000311789
[P->Y] 100.00000000256769
[C->X] 100.00000000000001
[C->Y] 99.99999999816593
